In [18]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [19]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1j32ukh0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-1j32ukh0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [20]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [21]:
!git clone https://github.com/nothings/stb.git
!cp stb/stb_image.h /usr/local/include/

fatal: destination path 'stb' already exists and is not an empty directory.


In [22]:
!pip install wget
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2024-04-09 14:31:29--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_write.h.1’

stb_image_write.h.1 100%[===================>]  69.55K  --.-KB/s    in 0.004s  

2024-04-09 14:31:29 (16.5 MB/s) - ‘stb_image_write.h.1’ saved [71221/71221]



# Final =========================>


In [23]:
%%writefile K1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>
using namespace std;

// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output ,unsigned int width, unsigned int height , const float* __restrict__ mask, int maskdim){

    int outRow = blockIdx.y * blockDim.y + threadIdx.y;
    int outCol = blockIdx.x * blockDim.x + threadIdx.x;
    int imageIndex = blockIdx.z * blockDim.z + threadIdx.z;

    if(outRow < height && outCol < width && imageIndex < batchSize) {

      float sum = 0;

        for(int maskRow = 0; maskRow < maskdim; ++maskRow){
           for(int maskCol = 0; maskCol < maskdim; ++maskCol){
              for(int maskdep = 0; maskdep < 3; ++maskdep){

                  int inRow = outRow -(maskdim/2) + maskRow;
                  int inCol = outCol -(maskdim/2) + maskCol;

                  if(inRow < height && inRow >= 0 && inCol < width && inCol >= 0){
                      sum += mask[maskdep * maskdim * maskdim + maskRow * maskdim + maskCol]
                      *input[imageIndex * width * height * 3 + maskdep * width * height + inRow * width + inCol];
                  }
             }
          }
        }
        if(sum < 0){
          sum = 0;
        } else if (sum > 255){
          sum = 255;
        }
        output[imageIndex * width * height + outRow * width + outCol] = sum;
    }

}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);


    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);


    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation
    dim3 threadsPerBlock(16, 16, 3);
    dim3 numBlocks((cols1-1) / threadsPerBlock.x + 1, (rows1-1) / threadsPerBlock.y + 1, (batch_size-1) / threadsPerBlock.z + 1);
    Imgcov3d<<<numBlocks, threadsPerBlock>>>(batch_size,d_images, d_outs,cols1,rows1,d_B, maskdim);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K1_2_31_2_33.cu


In [38]:
!chmod +x K1_2_31_2_33.cu
!nvcc  K1_2_31_2_33.cu -o K1_2_31_2_33

!./K1_2_31_2_33 "in" "out1" 4 "mask.txt"

maskdim 11
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg


In [25]:
!nvprof ./K1_2_31_2_33 "in" "out1" 4 "mask.txt"

maskdim 5
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg
==4042== NVPROF is profiling process 4042, command: ./K1_2_31_2_33 in out1 4 mask.txt
==4042== Profiling application: ./K1_2_31_2_33 in out1 4 mask.txt
==4042== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   64.51%  524.95us         2  262.48us  1.0240us  523.93us  [CUDA memcpy HtoD]
                   23.69%  192.80us         1  192.80us  192.80us  192.80us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   11.80%  96.000us         1  96.000us  96.000us  96.000us  [CUDA memcpy DtoH]
      API calls:   96.77%  73.099ms         3  24.366ms  7.0310us  72.977ms  cudaMalloc
                    2.34%  1.7664ms         3  588.80us  12.033us  1.0621ms  cudaMemcpy
                    0.43%  327.50us         3  109

In [65]:
!nvprof ./K1_2_31_2_33 "in1" "out1" 1 "mask.txt"

maskdim 5
cols 255
rows 255
image in1/cat (1).jpg
==15197== NVPROF is profiling process 15197, command: ./K1_2_31_2_33 in1 out1 1 mask.txt
==15197== Profiling application: ./K1_2_31_2_33 in1 out1 1 mask.txt
==15197== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   42.31%  66.335us         2  33.167us     704ns  65.631us  [CUDA memcpy HtoD]
                   39.27%  61.567us         1  61.567us  61.567us  61.567us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   18.41%  28.864us         1  28.864us  28.864us  28.864us  [CUDA memcpy DtoH]
      API calls:   98.65%  74.521ms         3  24.840ms  3.7410us  74.510ms  cudaMalloc
                    0.69%  520.87us         3  173.62us  8.4200us  326.39us  cudaMemcpy
                    0.26%  199.23us         1  199.23us  199.23us  199.23us  cudaLaunchKernel
                    0.19%  147.03us         3  49.010us  5.7810us 

In [26]:
%%writefile K2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;

// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height, const float* __restrict__ mask,int maskdim, int shared_width,int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;

    int x_i = x_o - maskdim/2;
    int y_i = y_o - maskdim/2;

    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    if (idx >= batchSize){
      return;
    }

  for (int d = 0; d < 3; d++) {

    if (x_i >= 0 && y_i >= 0 && x_i < width && y_i < height){

       sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + ty * shared_width + tx] =
        input[idx * width * height * 3 + d * height * width + y_i * width + x_i];
    }
    else{
       sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + ty * shared_width + tx] = 0;
    }
  }
    __syncthreads();

    float sum = 0.0;
    if(ty < tile_y && tx < tile_x) {

      for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[z_mask * maskdim * maskdim + y_mask * maskdim + x_mask] *
                    sharedTile[idx * shared_height * shared_width * 3 + z_mask * shared_height * shared_width + (ty + y_mask) * shared_width +(tx + x_mask)];

                }
             }
         }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

    if(y_o < height && x_o < width)
        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation

    dim3 threadsPerBlock(16, 16, 3);

    int tile_x = threadsPerBlock.x - (maskdim - 1);
    int tile_y = threadsPerBlock.y - (maskdim - 1);
    //int tile_z = threadsPerBlock.z - (batch_size - 1);
    int tile_z = threadsPerBlock.z;

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
    int z_blocks = (batch_size + tile_z - 1) / tile_z;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    //int shared_depth = tile_z + batch_size - 1;
    int shared_depth = batch_size;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1,d_B, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K2_2_31_2_33.cu


In [27]:
!chmod +x K2_2_31_2_33.cu
!nvcc  K2_2_31_2_33.cu -o K2_2_31_2_33

!./K2_2_31_2_33 "in" "out2" 4 "mask.txt"

maskdim 5
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg


In [28]:
!nvprof ./K2_2_31_2_33 "in" "out2" 4 "mask.txt"

maskdim 5
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg
==4130== NVPROF is profiling process 4130, command: ./K2_2_31_2_33 in out2 4 mask.txt
==4130== Profiling application: ./K2_2_31_2_33 in out2 4 mask.txt
==4130== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.85%  529.50us         2  264.75us  1.0240us  528.47us  [CUDA memcpy HtoD]
                   27.16%  240.29us         1  240.29us  240.29us  240.29us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   12.99%  114.91us         1  114.91us  114.91us  114.91us  [CUDA memcpy DtoH]
      API calls:   96.49%  70.586ms         3  23.529ms  5.0290us  70.465ms  cudaMalloc
                    2.55%  1.8641ms         3  621.36us  13.477us  1.1493ms  cudaMemcpy
                    0.45%  329.44us

In [64]:
!nvprof ./K2_2_31_2_33 "in1" "out2" 1 "mask.txt"

maskdim 5
cols 255
rows 255
image in1/cat (1).jpg
==14922== NVPROF is profiling process 14922, command: ./K2_2_31_2_33 in1 out2 1 mask.txt
==14922== Profiling application: ./K2_2_31_2_33 in1 out2 1 mask.txt
==14922== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.49%  89.567us         1  89.567us  89.567us  89.567us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   35.97%  66.432us         2  33.216us     704ns  65.728us  [CUDA memcpy HtoD]
                   15.54%  28.704us         1  28.704us  28.704us  28.704us  [CUDA memcpy DtoH]
      API calls:   98.65%  78.764ms         3  26.255ms  3.7240us  78.753ms  cudaMalloc
                    0.69%  549.93us         3  183.31us  9.1090us  340.87us  cudaMemcpy
                    0.29%  229.16us         1  229.16us  229.16us  229.16us  cudaLaunchKernel
                    0.18%  146.37us         3  48.7

In [35]:
%%writefile K3_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;


__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height, const float* __restrict__ mask,int maskdim, int shared_width,int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int bx = blockIdx.x;
    int by = blockIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;

    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    if (idx >= batchSize){
      return;
    }

    int num = ((width*height) - 1) /(shared_width*shared_height) + 1;

    for (int d=0; d< 3; d++){
      for(int f=0; f < num ; f++){

          int destTmp = tile_x * tile_y * f + ty * tile_x + tx;

          int dX = destTmp % shared_width;
          destTmp = destTmp / shared_width;
          int dY = destTmp % shared_height;

          int inputX = dX + (bx * tile_x) - (maskdim/2);
          int inputY = dY + (by * tile_y) - (maskdim/2);

          if(inputY >= 0 && inputY < height && inputX >= 0 && inputX < width){
                sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + dY * shared_width + dX]
                = input[idx * width * height * 3 + d * height * width + inputY * width + inputX];
          }
          else{
              sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + dY * shared_width + dX] = 0.0;
          }
      }
    }

    __syncthreads();

  float sum = 0;

    for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[z_mask * maskdim * maskdim + y_mask * maskdim + x_mask] *
                    sharedTile[idx * shared_height * shared_width * 3 + z_mask * shared_height * shared_width + (ty + y_mask) * shared_width +(tx + x_mask)];

                }
             }
         }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

    if(y_o < height && x_o < width){
        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation

    dim3 threadsPerBlock(16, 16, 3);

    int tile_x = threadsPerBlock.x;
    int tile_y = threadsPerBlock.y;
    //int tile_z = threadsPerBlock.z;
    int tile_z = threadsPerBlock.z;

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
    int z_blocks = (batch_size + tile_z - 1) / tile_z;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    //int shared_depth = tile_z + batch_size - 1;
    int shared_depth = batch_size;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1,d_B, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K3_2_31_2_33.cu


In [36]:
!chmod +x K3_2_31_2_33.cu
!nvcc  K3_2_31_2_33.cu -o K3_2_31_2_33

!./K3_2_31_2_33 "in" "out3" 4 "mask.txt"

maskdim 5
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg


In [37]:
!nvprof ./K3_2_31_2_33 "in" "out3" 4 "mask.txt"

maskdim 5
cols 255
rows 255
image in/you (1).jpg
cols 255
rows 255
image in/cat (1).jpg
cols 255
rows 255
image in/warda (1).jpg
cols 255
rows 255
image in/camel.jpg
==5975== NVPROF is profiling process 5975, command: ./K3_2_31_2_33 in out3 4 mask.txt
==5975== Profiling application: ./K3_2_31_2_33 in out3 4 mask.txt
==5975== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   84.59%  3.4652ms         1  3.4652ms  3.4652ms  3.4652ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   13.10%  536.83us         2  268.41us  1.0240us  535.80us  [CUDA memcpy HtoD]
                    2.30%  94.399us         1  94.399us  94.399us  94.399us  [CUDA memcpy DtoH]
      API calls:   92.25%  70.126ms         3  23.375ms  7.3160us  69.999ms  cudaMalloc
                    6.68%  5.0745ms         3  1.6915ms  13.450us  4.3413ms  cudaMemcpy
                    0.48%  362.03us

In [63]:
!nvprof ./K3_2_31_2_33 "in1" "out3" 1 "mask.txt"

maskdim 5
cols 255
rows 255
image in1/cat (1).jpg
==14379== NVPROF is profiling process 14379, command: ./K3_2_31_2_33 in1 out3 1 mask.txt
==14379== Profiling application: ./K3_2_31_2_33 in1 out3 1 mask.txt
==14379== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.01%  955.06us         1  955.06us  955.06us  955.06us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                    6.36%  66.752us         2  33.376us     704ns  66.048us  [CUDA memcpy HtoD]
                    2.63%  27.616us         1  27.616us  27.616us  27.616us  [CUDA memcpy DtoH]
      API calls:   97.48%  74.543ms         3  24.848ms  8.8370us  74.523ms  cudaMalloc
                    1.80%  1.3775ms         3  459.16us  11.529us  1.1832ms  cudaMemcpy
                    0.33%  251.32us         1  251.32us  251.32us  251.32us  cudaLaunchKernel
                    0.18%  135.93us         3  45.3

In [55]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import time
import numpy as np


# Define the mask tensor
file_path = "mask.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the lines of the file
    lines = file.readlines()

# Remove any leading or trailing whitespace characters from each line
lines = [line.strip() for line in lines]

# Extract the first line and assign it to a variable
MASKDIM= int(lines[0])
print("maskdim",MASKDIM)
# Extract the remaining lines and create a matrix
mask = [(line.split()) for line in lines[1:]]
# Convert the list to a numpy array
array_2d = np.array(mask, dtype=float)

# Convert the 2D array to a 3D array
mask = np.repeat(array_2d[np.newaxis, ...], 3, axis=0)

print(mask)

# Process the variables
print("MASKDIM:",MASKDIM )
print("Matrix:")
# for row in mask:
#     print(row)
print(mask)
mask = torch.from_numpy(mask)
print(mask)

mask = mask.unsqueeze(0).unsqueeze(0)
mask= mask.float()

# Read the RGB image using OpenCV
# image = cv2.imread('you.jpg')
# image = cv2.imread('camel.png')
image = cv2.imread('cat.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
#print(image[0])
image_array = np.array(image)
max_value = np.max(image_array)
max_value_arg = np.argmax(image_array)
print('===>', max_value, max_value_arg)

# Define the padding size
padding = (MASKDIM - 1) // 2
print(padding)

pad_height = (padding, padding)  # Pad 2 pixels on each side along height
pad_width = (padding, padding)  # Pad 2 pixels on each side along width
pad_channels = (0, 0)  # No padding along channels (RGB)

# Pad the image
padded_image = np.pad(image, (pad_height, pad_width, pad_channels), mode='constant', constant_values=0)  # Pad with zeros
print(padded_image.shape)
print("Data type of input_image:", padded_image.dtype)

# Convert the image to a PyTorch tensor
transform = transforms.ToTensor()
input_image = transform(padded_image)
print(input_image.shape)
print("Data type of input_image:", input_image.dtype)

input_image = input_image * 255

# Expand the dimensions of the input image to make it 3D
input_image = input_image.unsqueeze(0)
print(input_image.shape)

# Pad the input image
# padded_image = F.pad(input_image, (padding, padding, padding, padding, padding, padding))


# Perform the 3D convolution using PyTorch
# Start the timer
start_time = time.time()
output_tensor = F.conv3d(input_image, mask)
# Stop the timer
end_time = time.time()
output_tensor = output_tensor.to(torch.int)

# Remove the batch dimension and convert the output tensor to numpy array
output_array = output_tensor.squeeze(0).numpy()
# output_array=output_array*255
# Print the output array
print(output_array)

output_array = np.clip(output_array, 0, 255)  # Clip values to the range [0, 255]
print(output_array)
cv2.imwrite('pyimg_cpu.jpg', output_array.reshape(image.shape[0],image.shape[1],1))
# Calculate the execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution time:", execution_time, "seconds")

maskdim 5
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]]
MASKDIM: 5
Matrix:
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 

In [51]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import time
import numpy as np


# Define the mask tensor
file_path = "mask.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the lines of the file
    lines = file.readlines()

# Remove any leading or trailing whitespace characters from each line
lines = [line.strip() for line in lines]

# Extract the first line and assign it to a variable
MASKDIM = int(lines[0])
print("maskdim", MASKDIM)
# Extract the remaining lines and create a matrix
mask = [(line.split()) for line in lines[1:]]
# Convert the list to a numpy array
array_2d = np.array(mask, dtype=float)

# Convert the 2D array to a 3D array
mask = np.repeat(array_2d[np.newaxis, ...], 3, axis=0)

print(mask)

# Process the variables
print("MASKDIM:", MASKDIM)
print("Matrix:")
# for row in mask:
#     print(row)
print(mask)

# Convert the mask to a PyTorch tensor
mask = torch.from_numpy(mask).cuda()
print(mask)

mask = mask.unsqueeze(0).unsqueeze(0)
mask = mask.float()

# Read the RGB image using OpenCV
# image = cv2.imread('you.jpg')
# image = cv2.imread('camel.png')
image = cv2.imread('cat.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
# print(image[0])
image_array = np.array(image)
max_value = np.max(image_array)
max_value_arg = np.argmax(image_array)
print('===>', max_value, max_value_arg)

# Define the padding size
padding = (MASKDIM - 1) // 2
print(padding)

pad_height = (padding, padding)  # Pad 2 pixels on each side along height
pad_width = (padding, padding)  # Pad 2 pixels on each side along width
pad_channels = (0, 0)  # No padding along channels (RGB)

# Pad the image
padded_image = np.pad(image, (pad_height, pad_width, pad_channels), mode='constant', constant_values=0)  # Pad with zeros
print(padded_image.shape)
print("Data type of input_image:", padded_image.dtype)

# Convert the image to a PyTorch tensor
transform = transforms.ToTensor()
input_image = transform(padded_image)
print(input_image.shape)
print("Data type of input_image:", input_image.dtype)

input_image = input_image * 255

# Expand the dimensions of the input image to make it 3D
input_image = input_image.unsqueeze(0).cuda()
print(input_image.shape)

# Perform the 3D convolution using PyTorch
# Start the timer
start_time = time.time()
output_tensor = F.conv3d(input_image, mask)
# Stop the timer
end_time = time.time()
output_tensor = output_tensor.to(torch.int)

# Remove the batch dimension and convert the output tensor to a numpy array
output_array = output_tensor.squeeze(0).cpu().numpy()
# output_array=output_array*255
# Print the output array
print(output_array)

output_array = np.clip(output_array, 0, 255)  # Clip values to the range [0, 255]
print(output_array)
cv2.imwrite('pyimg_gpu.jpg', output_array.reshape(image.shape[0], image.shape[1], 1))
# Calculate the execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution time:", execution_time, "seconds")

maskdim 5
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]]
MASKDIM: 5
Matrix:
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 

In [46]:
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.8 MB/s eta 0:00:00


In [52]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import time
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# Define the mask tensor
file_path = "mask.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the lines of the file
    lines = file.readlines()

# Remove any leading or trailing whitespace characters from each line
lines = [line.strip() for line in lines]

# Extract the first line and assign it to a variable
MASKDIM = int(lines[0])
print("maskdim", MASKDIM)
# Extract the remaining lines and create a matrix
mask = [(line.split()) for line in lines[1:]]
# Convert the list to a numpy array
array_2d = np.array(mask, dtype=float)

# Convert the 2D array to a 3D array
mask = np.repeat(array_2d[np.newaxis, ...], 3, axis=0)

print(mask)

# Process the variables
print("MASKDIM:", MASKDIM)
print("Matrix:")
# for row in mask:
#     print(row)
print(mask)

# Convert the mask to a PyTorch tensor
mask = torch.from_numpy(mask)

print(mask)

mask = mask.unsqueeze(0).unsqueeze(0)
mask = mask.float()

# Read the RGB image using OpenCV
# image = cv2.imread('you.jpg')
# image = cv2.imread('camel.png')
image = cv2.imread('cat.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
# print(image[0])
image_array = np.array(image)
max_value = np.max(image_array)
max_value_arg = np.argmax(image_array)
print('===>', max_value, max_value_arg)

# Define the padding size
padding = (MASKDIM - 1) // 2
print(padding)

pad_height = (padding, padding)  # Pad 2 pixels on each side along height
pad_width = (padding, padding)  # Pad 2 pixels on each side along width
pad_channels = (0, 0)  # No padding along channels (RGB)

# Pad the image
padded_image = np.pad(image, (pad_height, pad_width, pad_channels), mode='constant', constant_values=0)  # Pad with zeros
print(padded_image.shape)
print("Data type of input_image:", padded_image.dtype)

# Convert the image to a PyTorch tensor
transform = transforms.ToTensor()
input_image = transform(padded_image)
print(input_image.shape)
print("Data type of input_image:", input_image.dtype)

input_image = input_image * 255

# Expand the dimensions of the input image to make it 3D
input_image = input_image.unsqueeze(0)
print(input_image.shape)

mask = mask.to(device)
input_image = input_image.to(device)

# Perform the 3D convolution using PyTorch
# Start the timer
start_time = time.time()
output_tensor = F.conv3d(input_image, mask)
# Stop the timer
end_time = time.time()
output_tensor = output_tensor.to(torch.int)

# Remove the batch dimension and convert the output tensor to a numpy array
output_array = output_tensor.squeeze(0).cpu().numpy()
# output_array=output_array*255
# Print the output array
print(output_array)

output_array = np.clip(output_array, 0, 255)  # Clip values to the range [0, 255]
print(output_array)
cv2.imwrite('pyimg_gpu.jpg', output_array.reshape(image.shape[0], image.shape[1], 1))
# Calculate the execution time
execution_time = end_time - start_time

# Print the execution time
print("Execution time:", execution_time, "seconds")

True
maskdim 5
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]]
MASKDIM: 5
Matrix:
[[[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]]

 [[0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0.013 0.013 0.013 0.013 0.013]
  [0